In [ ]:
import chess
import numpy as np
import chess.engine
import time
import chess.svg
import random
from random import shuffle
from IPython.display import SVG, display
import tracemalloc

In [ ]:
def generate_board():    
    def shuffle_word(word,fmt):
        word = word.lower()
        word = list(word)
        while True:
            shuffle(word)
            c = 0
            b = []
            for i in range(len(word)):
                if word[i] == 'k' and i <= 7:
                    c += 1
                if word[i] == 'b' and 8 <= i <= 15:
                    b.append(i+1)
                elif word[i] == 'b':
                    b.append(i)
            if len(b) ==2 and b[0] % 2 == 0 and b[1] % 2 == 1:
                c += 1
            if c == 2:
                break
        st = ''
        for i in range(len(word)):
            if i % 8 == 0 and i != 0:
                st += '/'
            if fmt == 'l' and word[i] != '1':
                st += word[i].lower()
            elif fmt == 'u' and word[i] != '1':
                st += word[i].upper()
            else:
                st += word[i]
        tmp = st.split('/')
        ret = ''
        for i in tmp:
            s = ''
            c = 0
            for ch in i:
                if ch == '1':
                    c += 1
                else:
                    if c != 0:
                        s += str(c)
                    s += ch
                    c = 0
            if c != 0:
                s += str(c)
            ret += s + '/'   
        return ret[:-1]

    board_t = chess.Board()
    arr = str(board_t.fen()).split(" ")
    inp = arr[0].split("/")
    inp[2] = '11111111'
    inp[-3] = '11111111'
    inp[0], inp[1], inp[2] = shuffle_word(inp[0]+inp[1]+inp[2],'l').split('/')
    inp[-1], inp[-2], inp[-3] = shuffle_word(inp[-1]+inp[-2]+inp[-3],'u').split('/')
    inp = '/'.join(inp)
    arr[0] = inp
    arr = ' '.join(arr)
    return chess.Board(arr)

In [ ]:
def static_eval(board):
    i = 0
    evaluation = 0
    x = True
    try:
        x = bool(board.piece_at(i).color)
    except AttributeError as e:
        x = x
    while i < 63:
        i += 1
        evaluation += np.log(get_piece_val(str(board.piece_at(i)))) if x else -np.log(get_piece_val(str(board.piece_at(i))))
    return evaluation

In [ ]:
def get_piece_val(piece):
    if(piece == None):
        return 0
    value = 0
    if piece == "P" or piece == "p":
        value = 10/100
    if piece == "N" or piece == "n":
        value = 30/100
    if piece == "B" or piece == "b":
        value = 30/100
    if piece == "R" or piece == "r":
        value = 50/100
    if piece == "Q" or piece == "q":
        value = 90/100
    if piece == 'K' or piece == 'k':
        value = 99/100
    return value+1

In [ ]:
def alpha_beta( board_instance, max_depth, current_depth, is_max_player, alpha, beta, nodes_per_depth,prev,markov):    
    if max_depth-current_depth in nodes_per_depth:
        nodes_per_depth[max_depth-current_depth] += 1
    else:
        nodes_per_depth[max_depth-current_depth] = 1
    
    if markov:
        leaf_node_score = static_eval(board_instance) * prev
    else:
        leaf_node_score = static_eval(board_instance)
    if current_depth == 0:
        return (leaf_node_score, nodes_per_depth)
    
    if is_max_player:
        best_score = -100000
        tmp = list(board_instance.legal_moves)
        inp = 3
        if (len(tmp) <= inp):
            inp = len(tmp)
        for legal_move in random.choices(tmp, k=inp):
            move =  chess.Move.from_uci(str(legal_move))
            board_instance.push(move)
            node_score, nodes_per_depth = alpha_beta(board_instance, max_depth, current_depth - 1, False, alpha, beta, nodes_per_depth,leaf_node_score,markov)
            best_score = max(best_score, node_score)
            board_instance.pop()
            alpha = max(alpha, best_score)
            if beta <= alpha:
                return (best_score, nodes_per_depth)       
        return (best_score, nodes_per_depth)
    else:
        best_score = 100000
        tmp = list(board_instance.legal_moves)
        inp = 3
        if (len(tmp) <= inp):
            inp = len(tmp)
        for legal_move in random.choices(tmp, k=inp):
            move =  chess.Move.from_uci(str(legal_move))
            board_instance.push(move)
            node_score, nodes_per_depth = alpha_beta(board_instance, max_depth, current_depth - 1, True, alpha, beta, nodes_per_depth,leaf_node_score,markov)
            best_score = min(best_score, node_score)
            board_instance.pop()
            beta = min(beta, best_score)
            if beta <= alpha:
                return (best_score, nodes_per_depth)
        return (best_score, nodes_per_depth) 

In [ ]:
def best_move_using_alpha_beta(board_instance, depth, is_max_player, alpha, beta, markov):
    best_move_score = -1000000
    best_move = None
    nodes_per_depth = None
    tmp = list(board_instance.legal_moves)
    inp = 3
    if (len(tmp) <= inp):
        inp = len(tmp)
    for legal_move in random.choices(tmp, k=inp):
        move = chess.Move.from_uci(str(legal_move))
        board_instance.push(move)
        move_score, nodes_per_depth = alpha_beta(board_instance,depth, depth, False, alpha, beta, {} ,1, markov)
        score = max(best_move_score, move_score)
        board_instance.pop()
        if score > best_move_score:
            best_move_score = score
            best_move = move
    return (best_move, nodes_per_depth)

In [ ]:
class Node:
    def __init__(self, move, parent=None, board=None):
        self.move = move
        self.parent = parent
        self.children = []
        self.visits = 0
        self.score = 0.0
        self.board = board

def uct(node):
    if node.visits == 0:
        return float('inf')
    return (node.score / node.visits) + 1.414 * math.sqrt(math.log(node.parent.visits) / node.visits)

def expand(node):
    legal_moves = list(node.board.legal_moves)
    for move in legal_moves:
        child_board = node.board.copy()
        child_board.push(move)
        child_node = Node(move, parent=node, board=child_board)
        node.children.append(child_node)

def simulate(node, max_depth):
    if max_depth == 0 or node.board.is_game_over():
        return static_eval(node.board)
    legal_moves = list(node.board.legal_moves)
    random_move = random.choice(legal_moves)
    node.board.push(random_move)
    score = simulate(node, max_depth - 1)
    node.board.pop()
    return score

def backpropagate(node, score):
    while node is not None:
        node.visits += 1
        node.score += score
        node = node.parent

def mcts(board, iterations=1000, max_depth=10):
    root = Node(None, board=board)

    for _ in range(iterations):
        current_node = root

        # Selection
        while not current_node.board.is_game_over() and not current_node.children:
            current_node = max(current_node.children, key=uct) if current_node.children else current_node
            expand(current_node)

        # Expansion
        if current_node.children:
            current_node = random.choice(current_node.children)

        # Simulation
        score = simulate(current_node, max_depth)

        # Backpropagation
        backpropagate(current_node, score)

    best_move = max(root.children, key=lambda x: x.visits).move
    return best_move

In [ ]:
def game_between_two_computer(depth_r,depth_m,num_of_games):
    points = {1:1, 2:3, 3:3, 4:5, 5:9, 6:0}
    w_win_count = 0
    b_win_count = 0
    draw_count = 0
    regular_win_count = 0
    markov_win_count = 0
    for gc in range(num_of_games):
        print("Game:",str(gc+1))
        w_move_count = 0
        b_move_count = 0
        w_point_count = 0
        b_point_count = 0
        board = generate_board()
        #board = chess.Board()
        n = 0
        prev_move = set()
        first = random.randint(0,1)
        second = 0
        if first == 0:
            second = 1
        if first == 0:
            print("White Strategy:","Regular Alpha Beta Pruning")
            print("Black Strategy:","Alpha Beta Pruning with Markov Chains")
        else:
            print("White Strategy:","Alpha Beta Pruning with Markov Chains")
            print("Black Strategy:","Regular Alpha Beta Pruning")
        max_r_bytes = 0
        max_m_bytes = 0
        while True:
            if n%2 == 0:
                #print("WHITE Turn")
                w_move_count += 1
                tracemalloc.start()
                if first == 0:
                    #move = mcts(board,iterations=200,max_depth=depth_r)
                    move, nodes_per_depth = best_move_using_alpha_beta(board, depth_r, False, -10000, 10000, first)
                    max_memory = tracemalloc.get_traced_memory()[1]
                    tracemalloc.stop()
                    max_r_bytes = max(max_r_bytes,max_memory)
                else:
                    move, nodes_per_depth = best_move_using_alpha_beta(board, depth_m, False, -10000, 10000, first)
                    max_memory = tracemalloc.get_traced_memory()[1]
                    tracemalloc.stop()
                    max_m_bytes = max(max_m_bytes,max_memory)
            else:
                #print("BLACK Turn")
                b_move_count += 1
                tracemalloc.start()
                if second == 0:
                    #move = mcts(board,iterations=200,max_depth=depth_r)
                    move, nodes_per_depth = best_move_using_alpha_beta(board, depth_r, True, -10000, 10000, second)
                    max_memory = tracemalloc.get_traced_memory()[1]
                    tracemalloc.stop()
                    max_r_bytes = max(max_r_bytes,max_memory)
                else:
                    move, nodes_per_depth = best_move_using_alpha_beta(board, depth_m, True, -10000, 10000, second)
                    max_memory = tracemalloc.get_traced_memory()[1]
                    tracemalloc.stop()
                    max_m_bytes = max(max_m_bytes,max_memory)
            
            if board.is_capture(move) and move and board.piece_at(move.to_square):
                piece = board.piece_at(move.to_square).piece_type
                if n % 2 == 0:
                    w_point_count += points[piece]
                else:
                    b_point_count += points[piece]
            board.push(move)
            
            outcome = board.outcome()
            if outcome:
                if outcome.winner == chess.WHITE:
                    w_win_count += 1
                    print("White Won")
                    if first == 0:
                        regular_win_count += 1
                    else:
                        markov_win_count += 1
                elif outcome.winner == chess.BLACK:
                    b_win_count += 1
                    print("Black Won")
                    if second == 0:
                        regular_win_count += 1
                    else:
                        markov_win_count += 1
                else:
                    draw_count += 1
                    print("Draw")
                break
            n += 1
        print("White Move Count:",w_move_count)
        print("White Point Count:",w_point_count)
        print("Black Move Count:",b_move_count)
        print("Black Point Count:",b_point_count)
        print()
    
    print("White Win Rate:",w_win_count/num_of_games)
    print("Black Win Rate:",b_win_count/num_of_games)
    print("Draw Rate:",draw_count/num_of_games)
    print("Regular Alpha Beta Pruning Win Count:",regular_win_count)
    print("Alpha Beta Pruning with Markov Chains Win Count:",markov_win_count)
    print("Max Memory Regular Alpha Beta Pruning Used in bytes:",max_r_bytes)
    print("Max Memory Alpha Beta Pruning with Markov Chains Used in bytes:",max_m_bytes)

In [ ]:
game_between_two_computer(2,2,1)